In [0]:
import numpy as np
import collections
from sklearn.metrics import confusion_matrix
from sklearn.feature_selection import mutual_info_classif
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, cross_val_predict
import time
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score, recall_score, f1_score, cohen_kappa_score
import matplotlib.pyplot as plt
import datetime
from xgboost import XGBClassifier

In [0]:
import os
os.listdir()

['.config', 'sample_data']

In [0]:
from google.colab import drive
drive.mount('/content/drive')
import os


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
os.chdir('/content/drive/My Drive/Courses/AI Course')
os.listdir()

['ch03.ipynb',
 'bankTestX.csv',
 'bankTestY.csv',
 'bankTrain.csv',
 'ESKIbank.csv',
 'd.png',
 'test.csv',
 'MySubmission.csv',
 'AI Classification.ipynb',
 'Unsupervised Learning.ipynb']

In [0]:
data= pd.read_csv('bankTrain.csv')

In [0]:
data.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,blue-collar,divorced,secondary,no,644,no,yes,cellular,17,nov,220,4,109,4,failure,no
1,52,admin.,married,primary,no,208,yes,no,cellular,17,apr,377,2,-1,0,unknown,no
2,34,technician,married,secondary,no,1641,yes,no,cellular,17,apr,380,1,-1,0,unknown,no
3,57,services,married,secondary,no,1433,no,no,telephone,21,nov,41,1,-1,0,unknown,no
4,57,technician,married,tertiary,no,2887,yes,no,cellular,11,feb,123,1,-1,0,unknown,no


In [0]:
from sklearn import preprocessing
for column_name in data.columns:
    if data[column_name].dtype == object:
        data[column_name] = data[column_name].astype('category')

In [0]:
data2 = pd.get_dummies(data, columns=['job', 'marital'])

In [0]:
data2.head()

,age,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,job_unemployed,job_unknown,marital_divorced,marital_married,marital_single
0,58,secondary,no,644,no,yes,cellular,17,nov,220,4,109,4,failure,no,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0
1,52,primary,no,208,yes,no,cellular,17,apr,377,2,-1,0,unknown,no,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2,34,secondary,no,1641,yes,no,cellular,17,apr,380,1,-1,0,unknown,no,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0
3,57,secondary,no,1433,no,no,telephone,21,nov,41,1,-1,0,unknown,no,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0
4,57,tertiary,no,2887,yes,no,cellular,11,feb,123,1,-1,0,unknown,no,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0


In [0]:
data2 = pd.get_dummies(data, columns=['job', 'marital','education', 'default','housing','loan','contact','month','poutcome'])

In [0]:
data2.head()

,age,balance,day,duration,campaign,pdays,previous,y,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,job_unemployed,job_unknown,marital_divorced,marital_married,marital_single,education_primary,education_secondary,education_tertiary,education_unknown,default_no,default_yes,housing_no,housing_yes,loan_no,loan_yes,contact_cellular,contact_telephone,contact_unknown,month_apr,month_aug,month_dec,month_feb,month_jan,month_jul,month_jun,month_mar,month_may,month_nov,month_oct,month_sep,poutcome_failure,poutcome_other,poutcome_success,poutcome_unknown
0,58,644,17,220,4,109,4,no,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0
1,52,208,17,377,2,-1,0,no,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,1,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,34,1641,17,380,1,-1,0,no,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,1,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,57,1433,21,41,1,-1,0,no,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,1,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1
4,57,2887,11,123,1,-1,0,no,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,1,0,0,1,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1


In [0]:
data.shape,data2.shape


((10000, 17), (10000, 52))

In [0]:
#data2.y=data2.y.apply(lambda x: x=='yes')

In [0]:
data2.head()

,age,balance,day,duration,campaign,pdays,previous,y,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,job_unemployed,job_unknown,marital_divorced,marital_married,marital_single,education_primary,education_secondary,education_tertiary,education_unknown,default_no,default_yes,housing_no,housing_yes,loan_no,loan_yes,contact_cellular,contact_telephone,contact_unknown,month_apr,month_aug,month_dec,month_feb,month_jan,month_jul,month_jun,month_mar,month_may,month_nov,month_oct,month_sep,poutcome_failure,poutcome_other,poutcome_success,poutcome_unknown
0,58,644,17,220,4,109,4,no,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0
1,52,208,17,377,2,-1,0,no,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,1,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,34,1641,17,380,1,-1,0,no,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,1,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,57,1433,21,41,1,-1,0,no,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,1,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1
4,57,2887,11,123,1,-1,0,no,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,1,0,0,1,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1


In [0]:
data2.y[110:120]

110     no
111     no
112    yes
113     no
114     no
115     no
116    yes
117     no
118    yes
119     no
Name: y, dtype: category
Categories (2, object): [no, yes]

In [0]:
data2.y.value_counts()

no     8854
yes    1146
Name: y, dtype: int64

In [0]:
train=data2[:7000]
test= data2[7000:]
train.shape, test.shape

((7000, 52), (3000, 52))

In [0]:
y_train= train.y 
X_train= train.drop(['y'], axis=1)
X_train.head()

,age,balance,day,duration,campaign,pdays,previous,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,job_unemployed,job_unknown,marital_divorced,marital_married,marital_single,education_primary,education_secondary,education_tertiary,education_unknown,default_no,default_yes,housing_no,housing_yes,loan_no,loan_yes,contact_cellular,contact_telephone,contact_unknown,month_apr,month_aug,month_dec,month_feb,month_jan,month_jul,month_jun,month_mar,month_may,month_nov,month_oct,month_sep,poutcome_failure,poutcome_other,poutcome_success,poutcome_unknown
0,58,644,17,220,4,109,4,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0
1,52,208,17,377,2,-1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,1,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,34,1641,17,380,1,-1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,1,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,57,1433,21,41,1,-1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,1,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1
4,57,2887,11,123,1,-1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,1,0,0,1,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1


In [0]:
X_train.shape, y_train.shape

((7000, 51), (7000,))

In [0]:
y_test= test.y 
X_test= test.drop(['y'], axis=1)
X_test.head()

,age,balance,day,duration,campaign,pdays,previous,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,job_unemployed,job_unknown,marital_divorced,marital_married,marital_single,education_primary,education_secondary,education_tertiary,education_unknown,default_no,default_yes,housing_no,housing_yes,loan_no,loan_yes,contact_cellular,contact_telephone,contact_unknown,month_apr,month_aug,month_dec,month_feb,month_jan,month_jul,month_jun,month_mar,month_may,month_nov,month_oct,month_sep,poutcome_failure,poutcome_other,poutcome_success,poutcome_unknown
7000,55,602,7,350,1,98,5,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0
7001,51,837,30,41,2,-1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,1,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1
7002,55,617,20,529,3,-1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
7003,27,255,12,123,1,182,4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,1,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0
7004,29,2,15,65,3,-1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,1,0,1,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1


In [0]:
X_test.shape, y_test.shape

((3000, 51), (3000,))

In [0]:
y_test.value_counts()

no     2649
yes     351
Name: y, dtype: int64

In [0]:
y_train.value_counts()

no     6205
yes     795
Name: y, dtype: int64

In [0]:
from sklearn.cluster import KMeans
import numpy as np
X = np.array([[1, 2], [10, 2], [1, 0], [10, 2], [10, 4], [10, 0]])

kmeans = KMeans(n_clusters=2, random_state=0).fit(X)
kmeans.labels_


array([1, 0, 1, 0, 0, 0], dtype=int32)

In [0]:
kmeans.predict([[10000, 100000], [12, 3] ])


array([0, 0], dtype=int32)

In [0]:
kmeans.cluster_centers_


array([[10.,  2.],
       [ 1.,  1.]])

In [0]:
kmeans = KMeans(n_clusters=10, random_state=0).fit(X_train)
kmeans.labels_

array([1, 8, 1, ..., 1, 9, 8], dtype=int32)

In [0]:
np.unique(kmeans.labels_, return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int32),
 array([ 389, 1626,    6,   82,   38,   17,  177,    1, 3877,  787]))

In [0]:
pred=kmeans.predict(X_test)

In [0]:
pred

array([1, 1, 1, ..., 9, 8, 8], dtype=int32)

In [0]:
np.unique(pred, return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int32),
 array([ 161,  731,    2,   27,   12,   17,  101,    2, 1615,  332]))